In [1]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv')
melb_df = melb_data.copy()
#удалим из таблицы столбцы
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
display(melb_df.head(1))
#введём признак MeanRoomsArea, который соответствует средней площади одной комнаты для каждого объекта
total_rooms = melb_df.Rooms + melb_df.Bathroom + melb_df.Bedroom
melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms
# AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#Посчитаем квадрат цены
price_square = melb_df['Price'] **2



,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,...,1,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019


Преобразование дат

In [11]:
#Преобразуем столбец Date в формат datetime, передав его в эту функцию:

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
years_sold = melb_df['Date'].dt.year
#print('Min year sold:', years_sold.min())
#print('Max year sold:', years_sold.max())
#print('Mode year sold:', years_sold.mode()[0])

#выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, 
# а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж
#melb_df['MonthSale'] = melb_df['Date'].dt.month
#melb_df['MonthSale'].value_counts(normalize=True)

#можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
#display(delta_days)
#Чтобы превратить количество дней из формата интервала в формат целого числа дней,
# можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days
#display(delta_days.dt.days)

#Давайте создадим признак возраста объекта недвижимости в годах на момент продажи.
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#год постройки удалим из таблицы
#melb_df = melb_df.drop(['YearBuilt'], axis=1)
display(melb_df.head(1))


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsArea,AreaRatio,AgeBuilding
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067,...,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,25.2,-0.231707,46


In [63]:
#2.3 Напишите функцию delete_columns(df, col=[]), которая удаляет столбцы из DataFrame и возвращает новую таблицу. 
#Если одного из указанных столбцов не существует в таблице, то функция должна возвращать None.
customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })
#display(customer_df.columns)

def delete_columns(df, col=[]):
    for elem in col:
        if elem not in df.columns:
            return None
    #df.drop(col, axis=1, inplace=True)
    return df.drop(col, axis=1)

#display(customer_df.columns.tolist())
#display(delete_columns(customer_df, col=['number', 'cust_age']))   

In [71]:
#2.4 Задан DataFrame countries_df, содержащий следующие столбцы: название страны,
# население (population) в миллионах человек и площадь страны (area) в квадратных километрах.
#Для каждой страны рассчитайте плотность населения (количество человек на квадратный километр).
#Затем по полученным данным рассчитайте среднее по плотностям населения в указанных странах. Ответ округлите до сотых.
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'area': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
#display(countries_df.head(2))
density = countries_df['population']*10**6 / countries_df['area']
#display(round(density.mean(),2))

In [12]:
#3.3 Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в 
# выходные (суббота и воскресенье), результат занесите в переменную weekend_count. 
# В качестве ответа введите результат вывода переменной weekend_count.

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
display(melb_df['WeekdaySale'].value_counts()[5]+melb_df['WeekdaySale'].value_counts()[6])
display(melb_df['WeekdaySale'].value_counts())


    

np.int64(12822)

WeekdaySale
5    11759
6     1063
0      597
1      133
3       28
Name: count, dtype: int64

In [4]:
#3.4 
nlo = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep = ',')
nlo['Time'] = pd.to_datetime(nlo['Time'], dayfirst=False)

# В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?
display(nlo['Time'].dt.year.mode()[0])

# Найдите средний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV).
# из столбца Time вытаскиваем дату
nlo['Date'] = nlo['Time'].dt.date

# вычитаем из строки столбца предыдущую по фильтру штата

date_diff = nlo[nlo['State'] == 'NV']['Date'].diff()

# из типа timedelta64[ns] получаем дни и находим среднее значение
date_diff_mean = date_diff.dt.days.mean()
print(date_diff_mean )




np.int32(1999)

68.92932862190813


 ### 4. Создание и преобразование столбцов с помощью функций

  ##### адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [13]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

print(melb_df['Address'].nunique())
#13378


# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type



2/119 Railway St N
9/400 Dandenong Rd
172 Danks St
13378


In [14]:
# Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). 
# В результате получим объект Series, который положим в переменную street_types:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

#Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. 
# Используемая функция обязательно должна иметь возвращаемое значение.

print(street_types.nunique())
print(street_types.value_counts())

# в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. 
# Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade.
#попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений
# Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.
#street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue','Bvd' if x == 'Boulevard','Pde' if x == 'Parade' else x)
street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue' else x)
street_types = street_types.apply(lambda x: 'Bvd' if x == 'Boulevard' else x)
street_types = street_types.apply(lambda x: 'Pde' if x == 'Parade' else x)
print(street_types.value_counts())
print(street_types.nunique())

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

56
Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Grn             4
Fairway         4
Mews            4
Gdns            4
Righi           3
Crossway        3
Esp             2
Ridge           2
Victoria        2
Crofts          2
Athol           1
Highway         1
Cove            1
Grange          1
Res             1
Terrace         1
Qy              1
Glade           1
Nook            1
Eyrie           1
Loop            1
Dell            1
East            1
Summit          1
Grand           1
Gra             1
Hts             1
Outlook         1
Woodland        1
Ave             1

In [15]:
#В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, 
# которые встречаются чаще всего, а остальные обозначим как 'other' (другие).
#Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series.
# Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index,
# а результат занесём в переменную popular_stypes:

popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])
print(melb_df['StreetType'].nunique())
display(melb_df['StreetType'].value_counts())
#melb_df = melb_df.drop('Address', axis=1)
#display(melb_df.dtypes)
#display(melb_df.head(1))


Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

11


StreetType
St       8012
Rd       2825
Ct        612
Dr        447
other     365
Av        361
Gr        311
Pde       226
Pl        169
Cr        152
Cl        100
Name: count, dtype: int64

In [16]:
# Задание 4.2
#Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, 
# если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.
#Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.
def get_weekend(weekday):
    if weekday in [5,6]:
        return 1
    return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean(), 0))

1081199.0


In [17]:
# 4.3
#Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: 
# оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
#Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson',
# больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.
#print(melb_df['SellerG'].value_counts())
print(melb_df['SellerG'].nunique())
sellers_top = melb_df['SellerG'].value_counts().nlargest(49).index
print(sellers_top)
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in sellers_top else 'other')
print(melb_df['SellerG'].nunique())
#print(melb_df['SellerG'].value_counts())

Nelson_other = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(Nelson_other)
a = 1.297709923664122 - Nelson_other

268
Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Purplebricks', 'Eview', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'Burnham', 'McDonald',
       'Moonee', 'LITTLE'],
      dtype='object', name='SellerG')
50
1.297709923664122


In [27]:
# 4.4
def get_experience(arg):
    word_list = arg.split(' ')
    month_list = ['месяц', 'месяца', 'месяцев']
    year_list = ['лет', 'года', 'год']
    year, month = 0, 0
    last_word = word_list[-1]
    if word_list[-1] in month_list:
        month = int(word_list[-2])
    if word_list[-3] in year_list:
        year = int(word_list[-4])
    if word_list[-1] in year_list:
        year = int(word_list[-2])
    
    return year*12 + month
    
arg = 'Опыт работы 1 месяц'
print(get_experience(arg))

import pandas as pd
test_series_1 = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])

test_series_2 = pd.Series([
    'Опыт работы 5 лет',
    'Опыт работы 5 месяцев',
    'Опыт работы 1 год 1 месяц',
    'Опыт работы 3 месяца',
    'Опыт работы 7 лет'
])
display(test_series_1.apply(get_experience))


1


0    99
1    41
2    21
3     3
4    72
dtype: int64

In [18]:
# определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
     # создаём кортеж (имя столбца, число уникальных значений, тип)
     item = (col, melb_df[col].nunique(), melb_df[col].dtypes)
     unique_list.append(item)
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(unique_list, columns=['Column_Name', 'Num_Unique', 'Type']).sort_values(by='Num_Unique',  ignore_index=True)

#melb_df = melb_df.drop('Address', axis=1)
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,WeekdaySale,5,int32
3,Method,5,object
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


In [19]:
# Сделаем преобразование столбцов к типу данных category:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns:
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude:
        melb_df[col] = melb_df[col].astype('category')


display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  category      
 1   Address        13580 non-null  object        
 2   Rooms          13580 non-null  int64         
 3   Type           13580 non-null  category      
 4   Price          13580 non-null  float64       
 5   Method         13580 non-null  category      
 6   SellerG        13580 non-null  category      
 7   Date           13580 non-null  datetime64[ns]
 8   Distance       13580 non-null  float64       
 9   Postcode       13580 non-null  int64         
 10  Bedroom        13580 non-null  int64         
 11  Bathroom       13580 non-null  int64         
 12  Car            13580 non-null  int64         
 13  Landsize       13580 non-null  float64       
 14  BuildingArea   13580 non-null  float64       
 15  YearBuilt      1358

None

In [20]:
#У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их.
# Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname:
print(melb_df['Regionname'].cat.categories)
display(melb_df['Regionname'].cat.codes.value_counts())
display(melb_df['Regionname'].value_counts())

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


5    4695
2    3890
6    2948
0    1471
4     450
1      53
3      41
7      32
Name: count, dtype: int64

Regionname
Southern Metropolitan         4695
Northern Metropolitan         3890
Western Metropolitan          2948
Eastern Metropolitan          1471
South-Eastern Metropolitan     450
Eastern Victoria                53
Northern Victoria               41
Western Victoria                32
Name: count, dtype: int64

In [21]:
#С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий.
# Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.
melb_df['Type'] = melb_df['Type'].cat.rename_categories({'u': 'unit','t': 'townhouse','h': 'house'})
display(melb_df['Type'].value_counts())

Type
house        9449
unit         3017
townhouse    1114
Name: count, dtype: int64

In [22]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [15]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Address        13580 non-null  object        
 2   Rooms          13580 non-null  int64         
 3   Type           13580 non-null  category      
 4   Price          13580 non-null  float64       
 5   Method         13580 non-null  category      
 6   SellerG        13580 non-null  category      
 7   Date           13580 non-null  datetime64[ns]
 8   Distance       13580 non-null  float64       
 9   Postcode       13580 non-null  int64         
 10  Bedroom        13580 non-null  int64         
 11  Bathroom       13580 non-null  int64         
 12  Car            13580 non-null  int64         
 13  Landsize       13580 non-null  float64       
 14  BuildingArea   13580 non-null  float64       
 15  YearBuilt      1358

None

In [27]:
# Преобразуйте признак Suburb следующим образом: оставьте в столбце только 119 наиболее популярных пригородов, остальные замените на 'other'.

suburb_pop = melb_df['Suburb'].value_counts().nlargest(119)
#melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in suburb_pop else 'other')
#melb_df['Suburb'] = melb_df['Suburb'].astype('object')
display(suburb_pop)
display(melb_df['Suburb'].nunique())
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in suburb_pop else 'other')
display(melb_df['Suburb'].nunique())
display(melb_df['Suburb'].value_counts())
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()
#melb_df = melb_df.drop('Address', axis=1)

Suburb
other              2215
Reservoir           359
Richmond            260
Bentleigh East      249
Preston             239
                   ... 
Greensborough        36
Chadstone            35
Caulfield North      35
Hughesdale           35
Mont Albert          34
Name: count, Length: 119, dtype: int64

119

119

Suburb
other              2215
Reservoir           359
Richmond            260
Bentleigh East      249
Preston             239
                   ... 
Greensborough        36
Chadstone            35
Caulfield North      35
Hughesdale           35
Mont Albert          34
Name: count, Length: 119, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  category      
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  category      
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  category      
 5   SellerG        13580 non-null  category      
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  YearBuilt      13580 non-null  category      
 15  CouncilArea    1221